## Finilizing Data Generation for PSR System

In [1]:
### Importing Libraries

import sys
print(sys.version)
import os
import numpy as np
import pandas as pd
import time

import matplotlib.pyplot as plt
WORKSPACE_PATH  = os.environ['WORKSPACE_PATH']
plt.style.use(WORKSPACE_PATH+'/ROMNet/romnet/extra/postprocessing/presentation.mplstyle')

from scipy.integrate import solve_ivp

3.8.12 (default, Aug 31 2021, 04:09:21) 
[Clang 12.0.5 (clang-1205.0.22.9)]


In [2]:
### Input Data

OutputDir  = WORKSPACE_PATH+'/ROMNet/Data/PSR_100Cases/'
TestDir    = WORKSPACE_PATH+'/ROMNet/Data/PSR_100Cases_Test/'
try:
    os.makedirs(OutputDir)
except OSError as e:
    pass
try:
    os.makedirs(TestDir)
except OSError as e:
    pass

tStratch           = 1.
SOLVER             = 'BDF'

ValidPerc          = 20.

FixedMinVal        = 1.e-10

In [3]:
### Retrieving Data

FileName = OutputDir + '/Orig/train/ext/Cleanvars.csv'
Vars     = pd.read_csv(FileName, header=None)
Vars     = list(Vars.to_numpy()[0,:])


FileName     = OutputDir + '/Orig/train/ext/RestVecTot.csv'
Data         = pd.read_csv(FileName, header=None)
RestVecTot   = np.log10(Data.to_numpy())
    
# FileName     = OutputDir+'/orig_data/t.csv'
# Data         = pd.read_csv(FileName, header=None)
# tVec         = Data.to_numpy()[:,0]

# FileName     = OutputDir + '/orig_data/yCleaned.csv'
# yMatOrig     = pd.read_csv(FileName, header=0)
# VarNames     = list(yMatOrig.columns)
# yMat         = yMatOrig.to_numpy()

FileName             = OutputDir+'/Orig/train/ext/yCleaned.csv'
Data                 = pd.read_csv(FileName, header=0)
Data.rename(columns={"# t": "t"}, inplace=True)
Data[Data.t == 0.].t = FixedMinVal
Data['log(t)']       = np.log(Data.t.to_numpy())
Data['log10(t)']     = np.log10(Data.t.to_numpy())
Data['log10(Rest)']  = RestVecTot
Data['Rest']         = 10.**RestVecTot
yMat                 = Data[Vars].to_numpy()


### Writing Input
InputVar     = ['t', 'log10(t)', 'Rest', 'log10(Rest)']
DataInput    = Data[InputVar]
DataInput.to_csv(OutputDir+'/Orig/train/ext/Input.csv', index=False)
tVec         = DataInput.t.to_numpy()


### Writing PCs
Data.to_csv(OutputDir+'/Orig/train/ext/Output.csv', index=False)

/usr/local/lib/python3.8/site-packages/pandas/core/generic.py:5159: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
<ipython-input-3-b4131e3eded2>:25: RuntimeWarning: divide by zero encountered in log
  Data['log(t)']       = np.log(Data.t.to_numpy())
<ipython-input-3-b4131e3eded2>:26: RuntimeWarning: divide by zero encountered in log10
  Data['log10(t)']     = np.log10(Data.t.to_numpy())


In [4]:
from sklearn.model_selection import train_test_split

n_points             = len(Data)

idx                  = np.arange(n_points)
train_idx, valid_idx = train_test_split(idx, test_size=ValidPerc/100, random_state=42)

n_valid              = len(valid_idx)
n_train              = len(train_idx)

try:
    os.makedirs(OutputDir+'/Orig/train/')
except:
    pass
try:
    os.makedirs(OutputDir+'/Orig/valid/')
except:
    pass

In [5]:
data_id    = 'pts'

try:
    os.makedirs(OutputDir+'/Orig/train/'+data_id+'/')
except:
    pass
try:
    os.makedirs(OutputDir+'/Orig/valid/'+data_id+'/')
except:
    pass

DataInput  = Data[['t', 'log10(t)', 'log(t)', 'log10(Rest)']]
DataInput.iloc[train_idx].to_csv(OutputDir+'/Orig/train/'+data_id+'/Input.csv', index=False)
DataInput.iloc[valid_idx].to_csv(OutputDir+'/Orig/valid/'+data_id+'/Input.csv', index=False)

DataOutput = Data[['t', 'log10(t)', 'log(t)'] + Vars]
DataOutput.iloc[train_idx].to_csv(OutputDir+'/Orig/train/'+data_id+'/Output.csv', index=False)
DataOutput.iloc[valid_idx].to_csv(OutputDir+'/Orig/valid/'+data_id+'/Output.csv', index=False)

In [6]:
data_id    = 'res'

try:
    os.makedirs(OutputDir+'/Orig/train/'+data_id+'/')
except:
    pass
try:
    os.makedirs(OutputDir+'/Orig/valid/'+data_id+'/')
except:
    pass

DataInput  = Data[['t', 'log10(t)', 'log10(t)', 'log10(Rest)']]
DataInput.iloc[train_idx].to_csv(OutputDir+'/Orig/train/'+data_id+'/Input.csv', index=False)
DataInput.iloc[valid_idx].to_csv(OutputDir+'/Orig/valid/'+data_id+'/Input.csv', index=False)

DataOutput       = Data[['t', 'log10(t)', 'log(t)'] + Vars]
DataOutput[Vars] = DataOutput[Vars].to_numpy() * 0.
DataOutput.iloc[train_idx].to_csv(OutputDir+'/Orig/train/'+data_id+'/Output.csv', index=False)
DataOutput.iloc[valid_idx].to_csv(OutputDir+'/Orig/valid/'+data_id+'/Output.csv', index=False)

<ipython-input-6-47a756bd24f1>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DataOutput[Vars] = DataOutput[Vars].to_numpy() * 0.
/usr/local/lib/python3.8/site-packages/pandas/core/frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.iloc._setitem_with_indexer((slice(None), indexer), value)
/usr/local/lib/python3.8/site-packages/pandas/core/frame.py:3034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.

In [7]:
data_id = 'ics'

try:
    os.makedirs(OutputDir+'/Orig/train/'+data_id+'/')
except:
    pass
try:
    os.makedirs(OutputDir+'/Orig/valid/'+data_id+'/')
except:
    pass

Data_ics                     = Data[Data['t'] == 0.]
n_points_ics                 = len(Data_ics)
idx_ics                      = np.arange(n_points_ics)
train_idx_ics, valid_idx_ics = train_test_split(idx_ics, test_size=ValidPerc/100, random_state=42)

DataInput      = Data_ics[['t', 'log10(t)', 'log(t)', 'log10(Rest)']]
DataInput.iloc[train_idx_ics].to_csv(OutputDir+'/Orig/train/'+data_id+'/Input.csv', index=False)
DataInput.iloc[valid_idx_ics].to_csv(OutputDir+'/Orig/valid/'+data_id+'/Input.csv', index=False)

DataOutput       = Data_ics[['t', 'log10(t)', 'log(t)'] + Vars]
DataOutput.iloc[train_idx_ics].to_csv(OutputDir+'/Orig/train/'+data_id+'/Output.csv', index=False)
DataOutput.iloc[valid_idx_ics].to_csv(OutputDir+'/Orig/valid/'+data_id+'/Output.csv', index=False)

In [8]:
# ### Retrieving Data

# Vars = []
# for iVars in range(1,NVarsRed+1):
#     Vars.append('PC_'+str(iVars))

# SVars = []
# for iVars in range(1,NVarsRed+1):
#     SVars.append('SPC_'+str(iVars))

# FileName     = TestDir + '/orig_data/RestVecTot.csv'
# Data         = pd.read_csv(FileName, header=None)
# RestVecTot   = np.log10(Data.to_numpy())
    
# # FileName     = TestDir+'/orig_data/t.csv'
# # Data         = pd.read_csv(FileName, header=None)
# # tVec         = Data.to_numpy()[:,0]

# # FileName     = TestDir + '/orig_data/yCleaned.csv'
# # yMatOrig     = pd.read_csv(FileName, header=0)
# # VarNames     = list(yMatOrig.columns)
# # yMat         = yMatOrig.to_numpy()

# FileName             = TestDir+'/'+str(NVarsRed)+'PC/PC.csv'
# Data                 = pd.read_csv(FileName, header=0)
# Data['log10(t)']     = np.log10(Data.t.to_numpy()+1.e-15)
# Data['log10(Rest)']  = RestVecTot
# Data['Rest']         = 10.**RestVecTot
# yMat_pca             = Data[Vars].to_numpy()

# FileName             = TestDir+'/'+str(NVarsRed)+'PC/PCSource.csv'
# DataS                = pd.read_csv(FileName, header=0)
# DataS['log10(t)']    = np.log10(Data.t.to_numpy()+1.e-15)
# DataS['log10(Rest)'] = RestVecTot
# DataS['Rest']        = 10.**RestVecTot
# ySource_pca          = DataS[SVars].to_numpy()


# ### Writing Input
# InputVar     = ['t', 'log10(t)', 'Rest', 'log10(Rest)']
# DataInput    = Data[InputVar]
# DataInput.to_csv(TestDir+'/'+str(NVarsRed)+'PC/Input.csv', index=False)
# tVec         = DataInput.t.to_numpy()


# ### Writing PCs
# Data.to_csv(TestDir+'/'+str(NVarsRed)+'PC/Output.csv', index=False)


# ### Normalizing PCs' Source Terms
# ySourcee_pca = (ySource_pca - yMin)/yRange
# #ySourcee_pca = (ySource_pca - yMin)/yRange[0]

# for iPC in range(NVarsRed):
#     DataS['SPC_'+str(iPC+1)+'_Scaled'] = ySourcee_pca[:,iPC]
# DataS.to_csv(TestDir+'/'+str(NVarsRed)+'PC/dOutput.csv', index=False)